In [1]:
from tardis import run_tardis
import tardis
import plotly
from plotly.graph_objs import graph_objs as go
import ipywidgets as widgets
from collections import defaultdict

In [2]:
from pylab import *

cmap = cm.get_cmap('jet', 20)
colors = []
for i in range(cmap.N):
    rgb = cmap(i)[:3]
    colors.append(matplotlib.colors.rgb2hex(rgb))

In [3]:
# making subplots
fig = go.FigureWidget().set_subplots(1,2, shared_xaxes=True,)
fig.add_scatter(row = 1, col = 1)
fig.add_scatter(row = 1, col = 2)

# updating axes 
fig['layout']['yaxis2']['title'] = r"$W$"
fig['layout']['xaxis2']['title'] = r"$Shell~~Velocity$"

fig['layout']['yaxis']['title'] = r"$T_{rad}\ [K]$"
fig['layout']['xaxis']['title'] = r"$Shell~~Velocity$"


fig['layout']['yaxis']['range'] = [9000, 14000]
fig['layout']['yaxis']['tickformat'] =  "g"
fig['layout']['xaxis2']['tickformat'] =  "g"
fig['layout']['xaxis']['tickformat'] =  "g"
fig['layout']['xaxis2']['matches'] =  "x"

fig.update_layout(
    margin=dict(
        l=10,
        r=135,
        b=80,
        t=25,
        pad=0
    )
)

plasma_fig = fig.update_layout(height = 580)


In [4]:

line_colors = ["orangered", 'lightseagreen', 'indigo']

luminosities = ["Emitted", "Absorbed", "Requested"]
X = list(range(1,21))


fig = go.FigureWidget().set_subplots(
        3,1, 
        shared_xaxes=True, 
        vertical_spacing = 0.05,
        row_heights=[0.15, 0.7, 0.15]
    )

for luminosity, line_color in zip(luminosities, line_colors):
    fig.add_scatter(
        name = luminosity + "<br>Luminosity", 
        row = 2, 
        col = 1,
        mode="lines",
        marker_color=line_color, 
        legendgroup = luminosity, 
#         marker_line_width=1.5, 
        opacity=0.6
    )
fig.add_scatter(
    name = "Residual<br>Luminosity",
    row = 3, 
    col = 1, 
    mode="lines",
    
    marker_color='cornflowerblue', 
    opacity=0.7

)
fig.add_scatter(
    name = "Next Inner<br>Boundary<br>Temperature",
    row = 1, 
    col = 1, 
    mode="lines",
    
    marker_color='crimson', 
    opacity=0.7

)


fig = fig.update_layout(
    xaxis = dict(
        range = [0, 21],
        dtick = 2,
        
    ),
    xaxis2 = dict(
        matches = "x",
        range = [0, 21],
        dtick = 2,
    ),
    xaxis3 = dict(
        matches = "x",
        title = r"$\mbox{Iteration Number}$", 
        dtick = 2,
    ),
    
    yaxis = dict(
        title = r"$\mbox{T}_{inner}$",
        automargin = True,
        side = "top",
        tickformat="g",
        exponentformat = "e",
    ),
    yaxis2 = dict(
        exponentformat = "e",
        title = r"$\mbox{Luminosity}~(erg~sec^{-1})$",
        title_font_size = 13,
        automargin = True
    ),
    yaxis3 = dict(
        exponentformat = "e",
        title = r"$~~\mbox{Residual}\\\mbox{Luminosity(%)}$", 
        title_font_size = 12,
        automargin = True
    ),
    
    legend_tracegroupgap = 0,
    hidesources = True,
    height = 600,
    hoverlabel_align = 'right',
    legend_title_text = "Luminosity",
    margin=dict(
        l=10,
        r=10,
        b=10,
        t=25,
        pad=0
    ),
    hoverlabel_font_color ="white"
)
luminosity_fig = fig


In [5]:
widgets.VBox([plasma_fig, luminosity_fig], layout=widgets.Layout( height='1000px'))

    'data': [{'type': 'scatter', 'uid': '9b5ed1ef-dcc2-401a-b1c3-39cf40ea464b', …

In [6]:
traces = defaultdict(list)
i=0
ic = 0

In [7]:

def update_t_rad(sim):
    """
    adds another scatter trace of T_rad vs Shell Velocity with dark blue line color
    """
    
    plasma_fig.add_scatter(
        x = [item/100000 for item in sim.model.velocity.value.tolist()], 
        y = sim.model.t_rad.value.tolist(), 
        line_color = colors[ic], 
        row = 1, 
        col = 1, 
        name = ic+1,
        legendgroup=f"group-{ic}", 
        marker=dict(symbol="diamond", size=3),
        showlegend = False,
    )

    
def update_w(sim):
    """
    adds another scatter trace of W vs Shell Velocity with dark blue line color
    """
    plasma_fig.add_scatter(
        x = [item/100000 for item in sim.model.velocity.value.tolist()], 
        y = sim.model.w.tolist(), 
        line_color= colors[ic], 
        row = 1, 
        col = 2, 
        legendgroup=f"group-{ic}",
        marker=dict(symbol="diamond", size=3),
        name = ic+1,
    )

    
def fix_colors(index):
    """
    changes line color to light blue
    """
    plasma_fig['data'][index]['line']['color']=colors[ic-1]
    


    
def update_traces(sim):

    global traces, i, ic

    emitted_luminosity = sim.runner.calculate_emitted_luminosity(
        sim.luminosity_nu_start, sim.luminosity_nu_end
    )
    reabsorbed_luminosity = sim.runner.calculate_reabsorbed_luminosity(
        sim.luminosity_nu_start, sim.luminosity_nu_end
    )

    traces["Emitted"].append(emitted_luminosity.value)
    traces["Absorbed"].append(reabsorbed_luminosity.value)
    traces["Requested"].append(sim.luminosity_requested.value)
    traces["t_inner"].append(sim.model.t_inner.value)


    with luminosity_fig.batch_update():
        for index, luminosity in zip(range(3), luminosities):
            luminosity_fig.data[index].x = X
            luminosity_fig.data[index].y = traces[luminosity]
            luminosity_fig.data[index].hovertemplate = '<b>%{y:.2g}</b>'+'<br>at X = %{x}<br>'
    
        y = [((emitted-requested)*100)/requested for emitted, requested in zip(traces["Emitted"],traces["Requested"])]
        luminosity_fig.data[-2].x = X
        luminosity_fig.data[-2].y = y
        luminosity_fig.data[-2].hovertemplate='Residual Luminosity: %{y:.2f}% at X = %{x:,.0f}<extra></extra>'
        
        


        luminosity_fig.data[-1].x = X
        luminosity_fig.data[-1].y = traces["t_inner"]
        luminosity_fig.data[-1].hovertemplate='Next Inner Body Temperature: %{y:.2f} at X = %{x:,.0f}<extra></extra>'
        

    i = i+2
    
    fix_colors(i-1)
    fix_colors(i-2)
        
    update_t_rad(sim)
    update_w(sim)
    ic+=1
    
    plasma_fig.update_layout(
        legend_tracegroupgap = 0,
        legend_title_text = "iterations",
        hidesources = True
    )
    
    

In [8]:
sim = run_tardis('tardis_example.yml', 
                 simulation_callbacks=[[update_traces]])

[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/atharva/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
[py.warnings         ][WARNING]  /home/atharva/.conda/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning:

divide by zero encountered in double_scalars

 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity

[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11567.208184  10569.747403  0.504178  0.442290
	5      11653.124121  11099.194259  0.203042  0.153253
	10     11425.955010  10809.397890  0.125249  0.097256
	15     11082.155938  10454.841299  0.094028  0.073809

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/atharva/.conda/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning:

divide by zero encountered in double_scalars

 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.33305e+43 erg / s Luminosity absorbed = 4.33011e+42 erg / s Luminosity requested = 1.05928e+43 erg

[tardis.simulation.base][INFO   ]  t_inner 11175.750 K -- next t_inner 10152.603 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 15/20 (base.py:323)
[py.warnings         ][WARNING]  /home/atharva/.conda/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning:

divide by zero encountered in double_scalars

 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.75331e+42 erg / s Luminosity absorbed = 2.81319e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11437.188130  10637.774893  0.496749  0.452668
	5      11571.106856  11002.647927  0.196714  0.166197
	10     11378.137918  10832.042108  0.120954  0.101284
	15     10957.211814  10350.602742  0.092896  0.079

In [9]:
plasma_fig_out = widgets.Output()
lum_fig_out = widgets.Output()
widgets.VBox([plasma_fig_out, lum_fig_out])

In [10]:
with plasma_fig_out:
    plasma_fig.show(renderer = "notebook_connected")
with lum_fig_out:
    luminosity_fig.show(renderer = "notebook_connected")

In [11]:
# sharing notebooks take 2
